# Consultas SQL Analíticas con cx_Oracle

En este notebook exploraremos consultas SQL analíticas directas utilizando cx_Oracle como complemento a los enfoques MDX. Aprenderemos cuándo usar SQL directo vs MDX para obtener máximo rendimiento.

## 1. Configuración y Conexión Oracle

Configuramos la conexión directa a Oracle usando cx_Oracle:

In [1]:
import cx_Oracle
import pandas as pd
import os
import time
from datetime import datetime

# Configuración de conexión Oracle
def conectar_oracle():
    """Establece conexión directa con Oracle"""
    try:
        dsn = cx_Oracle.makedsn(
            host=os.getenv('ORACLE_HOST', 'localhost'),
            port=1521,
            service_name=os.getenv('ORACLE_SERVICE', 'XEPDB1')
        )
        
        conexion = cx_Oracle.connect(
            user=os.getenv('ORACLE_USER', 'C##DM_ACADEMICO'),
            password=os.getenv('ORACLE_PASSWORD', 'YourPassword123'),
            dsn=dsn
        )
        
        print("✅ Conexión Oracle establecida exitosamente")
        print(f"📊 Versión Oracle: {conexion.version}")
        return conexion
        
    except Exception as e:
        print(f"❌ Error conectando a Oracle: {e}")
        return None

# Establecer conexión
conexion = conectar_oracle()

✅ Conexión Oracle establecida exitosamente
📊 Versión Oracle: 21.3.0.0.0


## 2. Consultas SQL Analíticas Básicas

Implementamos funciones analíticas SQL para análisis directo de datos:

In [2]:
# 🚀 RANKING Y FUNCIONES DE VENTANA
def ranking_centros_por_matriculas():
    """Ranking de centros usando WINDOW functions"""
    sql = """
    SELECT 
        c.NOMBRE_CENTRO,
        COUNT(*) as total_matriculas,
        RANK() OVER (ORDER BY COUNT(*) DESC) as ranking,
        DENSE_RANK() OVER (ORDER BY COUNT(*) DESC) as ranking_denso,
        PERCENT_RANK() OVER (ORDER BY COUNT(*)) as percentil,
        ROW_NUMBER() OVER (ORDER BY COUNT(*) DESC) as fila
    FROM F_MATRICULA m
    JOIN D_CENTRO c ON m.ID_CENTRO = c.ID_CENTRO
    GROUP BY c.NOMBRE_CENTRO
    ORDER BY ranking
    """
    
    resultado = pd.read_sql(sql, conexion)
    print("🏆 Ranking de Centros por Matrículas:")
    return resultado

# Ejecutar consulta
ranking_centros = ranking_centros_por_matriculas()
print(ranking_centros.head(10))

🏆 Ranking de Centros por Matrículas:
                                       NOMBRE_CENTRO  TOTAL_MATRICULAS  \
0            Facultad de Traducción e Interpretación                20   
1                   Centro de Estudios Superiores 11                16   
2  Escuela Técnica Superior de Ingeniería Industrial                16   
3                             Facultad de Medicina 3                15   
4    Facultad de Ciencias Económicas y Empresariales                14   
5              Escuela Universitaria de Enfermería 4                14   
6                             Facultad de Enfermería                13   
7                   Centro Internacional de Posgrado                13   
8        Escuela Técnica Superior de Arquitectura 15                12   
9                      Centro de Estudios Superiores                12   

   RANKING  RANKING_DENSO  PERCENTIL  FILA  
0        1              1   1.000000     1  
1        2              2   0.959184     3  
2        2   

C:\Users\theca\AppData\Local\Temp\ipykernel_9848\2075321010.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  resultado = pd.read_sql(sql, conexion)


In [4]:
# 🚀 ANÁLISIS TEMPORAL CON LAG/LEAD
def analisis_temporal_rendimiento():
    """Análisis temporal con funciones LAG y LEAD"""
    sql = """
    WITH datos_anuales AS (
        SELECT 
            ca.ID_CURSO_ACADEMICO_NK as año,
            AVG(r.NOTA_NUMERICA) as nota_promedio,
            COUNT(CASE WHEN r.FLG_SUPERADA = 1 THEN 1 END) * 100.0 / COUNT(*) as tasa_exito_promedio
        FROM F_RENDIMIENTO r
        JOIN D_CURSO_ACADEMICO ca ON r.ID_CURSO_ACADEMICO = ca.ID_CURSO_ACADEMICO
        WHERE r.NOTA_NUMERICA IS NOT NULL
        GROUP BY ca.ID_CURSO_ACADEMICO_NK
    )
    SELECT 
        año,
        ROUND(nota_promedio, 2) as nota_actual,
        ROUND(LAG(nota_promedio) OVER (ORDER BY año), 2) as nota_año_anterior,
        ROUND(
            (nota_promedio - LAG(nota_promedio) OVER (ORDER BY año)) * 100.0 /
            LAG(nota_promedio) OVER (ORDER BY año), 2
        ) as variacion_porcentual,
        ROUND(tasa_exito_promedio, 2) as tasa_exito
    FROM datos_anuales
    ORDER BY año
    """
    
    resultado = pd.read_sql(sql, conexion)
    print("📈 Análisis Temporal de Rendimiento:")
    return resultado

# Ejecutar análisis temporal
analisis_temporal = analisis_temporal_rendimiento()
print(analisis_temporal)

📈 Análisis Temporal de Rendimiento:
     AÑO  NOTA_ACTUAL  NOTA_AÑO_ANTERIOR  VARIACION_PORCENTUAL  TASA_EXITO
0   2010         4.84                NaN                   NaN       49.18
1   2011         4.69               4.84                 -3.07       48.44
2   2012         5.28               4.69                 12.72       58.21
3   2013         4.05               5.28                -23.33       33.90
4   2014         4.98               4.05                 23.00       50.88
5   2015         4.67               4.98                 -6.34       39.08
6   2016         5.48               4.67                 17.42       54.79
7   2017         4.72               5.48                -13.85       51.47
8   2018         5.06               4.72                  7.17       45.10
9   2019         5.16               5.06                  1.90       50.00
10  2020         5.29               5.16                  2.53       60.71
11  2021         5.51               5.29                  4.17  

C:\Users\theca\AppData\Local\Temp\ipykernel_9848\3948717361.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  resultado = pd.read_sql(sql, conexion)


## 3. Consultas con PARTITION BY

Utilizamos PARTITION BY para análisis segmentado:

In [18]:
# 🚀 ANÁLISIS CON PARTITION BY QUE FUNCIONA
def analisis_partition_by():
    """Análisis usando PARTITION BY con datos reales"""
    
    print("\n🎯 ANÁLISIS SEGMENTADO CON PARTITION BY:")
    print("=" * 50)
    
    try:
        # Análisis por centro y año con PARTITION BY
        resultado = pd.read_sql("""
        WITH datos_centro_año AS (
            SELECT 
                c.NOMBRE_CENTRO,
                c.NOMBRE_TIPO_CENTRO,
                m.ID_CURSO_ACADEMICO_NK as año,
                COUNT(*) as matriculas,
                ROUND(AVG(m.EDAD), 1) as edad_promedio,
                SUM(m.CREDITOS) as creditos_totales
            FROM F_MATRICULA m
            JOIN D_CENTRO c ON m.ID_CENTRO = c.ID_CENTRO
            WHERE m.EDAD IS NOT NULL AND m.CREDITOS IS NOT NULL
            GROUP BY c.NOMBRE_CENTRO, c.NOMBRE_TIPO_CENTRO, m.ID_CURSO_ACADEMICO_NK
        )
        SELECT 
            NOMBRE_CENTRO,
            NOMBRE_TIPO_CENTRO,
            año,
            matriculas,
            edad_promedio,
            creditos_totales,
            RANK() OVER (PARTITION BY NOMBRE_TIPO_CENTRO ORDER BY matriculas DESC) as rank_por_tipo,
            RANK() OVER (PARTITION BY año ORDER BY matriculas DESC) as rank_por_año,
            ROUND(
                matriculas * 100.0 / SUM(matriculas) OVER (PARTITION BY NOMBRE_TIPO_CENTRO), 2
            ) as porcentaje_en_tipo,
            ROUND(
                matriculas * 100.0 / SUM(matriculas) OVER (PARTITION BY año), 2
            ) as porcentaje_en_año,
            ROUND(AVG(matriculas) OVER (PARTITION BY NOMBRE_TIPO_CENTRO), 2) as promedio_tipo
        FROM datos_centro_año
        ORDER BY año DESC, rank_por_año
        FETCH FIRST 20 ROWS ONLY
        """, conexion)
        
        print("✅ ANÁLISIS SEGMENTADO:")
        print(resultado.to_string())
        
        return resultado
        
    except Exception as e:
        print(f"❌ Error en PARTITION BY: {e}")
        return None

# Ejecutar análisis con PARTITION BY
resultado_partition = analisis_partition_by()


🎯 ANÁLISIS SEGMENTADO CON PARTITION BY:
✅ ANÁLISIS SEGMENTADO:
                                                   NOMBRE_CENTRO        NOMBRE_TIPO_CENTRO   AÑO  MATRICULAS  EDAD_PROMEDIO  CREDITOS_TOTALES  RANK_POR_TIPO  RANK_POR_AÑO  PORCENTAJE_EN_TIPO  PORCENTAJE_EN_AÑO  PROMEDIO_TIPO
0                        Facultad de Traducción e Interpretación           Centro Adscrito  2024           3           33.7              30.3              1             1                3.90               7.89           1.40
1             Escuela Técnica Superior de Ingeniería Informática   Instituto Universitario  2024           3           24.7              18.8              1             1                4.55               7.89           1.40
2                                         Facultad de Derecho 17           Centro Adscrito  2024           3           32.7              20.3              1             1                3.90               7.89           1.40
3                                   

C:\Users\theca\AppData\Local\Temp\ipykernel_9848\3899326491.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  resultado = pd.read_sql("""


## 4. Comparación SQL vs MDX

Comparamos el mismo análisis usando SQL directo vs MDX para entender cuándo usar cada enfoque:

In [22]:
# 🚀 COMPARACIÓN DIRECTA: SQL vs MDX
def comparacion_rendimiento_por_centro():
    """Misma consulta en SQL - luego comparamos con MDX"""
    
    # Enfoque SQL directo
    inicio_sql = time.time()
    
    print("🔍 Verificando estructura de tablas...")
    
    # Verificar estructura de D_CENTRO
    try:
        centro_cols = pd.read_sql("SELECT * FROM D_CENTRO WHERE ROWNUM <= 1", conexion)
        print(f"📊 Columnas D_CENTRO: {centro_cols.columns.tolist()}")
    except Exception as e:
        print(f"Error verificando D_CENTRO: {e}")
    
    # Verificar estructura de D_ESTUDIO  
    try:
        estudio_cols = pd.read_sql("SELECT * FROM D_ESTUDIO WHERE ROWNUM <= 1", conexion)
        print(f"📚 Columnas D_ESTUDIO: {estudio_cols.columns.tolist()}")
    except Exception as e:
        print(f"Error verificando D_ESTUDIO: {e}")
    
    # Consulta corregida usando las columnas que existen
    sql_query = """
    SELECT 
        dc.NOMBRE_CENTRO as nombre_centro,
        dc.NOMBRE_TIPO_CENTRO as tipo_centro,
        COUNT(fr.ID_EXPEDIENTE_ACADEMICO) as total_registros,
        ROUND(AVG(fr.NOTA_NUMERICA), 2) as nota_promedio,
        ROUND(
            COUNT(CASE WHEN fr.FLG_SUPERADA = 1 THEN 1 END) * 100.0 / COUNT(*), 2
        ) as tasa_exito_promedio,
        ROUND(
            AVG(fr.NOTA_NUMERICA) * 
            (COUNT(CASE WHEN fr.FLG_SUPERADA = 1 THEN 1 END) * 100.0 / COUNT(*)) / 100, 2
        ) as indice_rendimiento
    FROM F_RENDIMIENTO fr
    JOIN D_CENTRO dc ON fr.ID_CENTRO = dc.ID_CENTRO
    WHERE fr.NOTA_NUMERICA IS NOT NULL
    GROUP BY dc.NOMBRE_CENTRO, dc.NOMBRE_TIPO_CENTRO
    HAVING AVG(fr.NOTA_NUMERICA) > 5.0
    ORDER BY indice_rendimiento DESC
    FETCH FIRST 20 ROWS ONLY
    """
    
    try:
        resultado_sql = pd.read_sql(sql_query, conexion)
        tiempo_sql = time.time() - inicio_sql
        
        print(f"⚡ SQL ejecutado en: {tiempo_sql:.3f} segundos")
        print(f"📊 Registros obtenidos: {len(resultado_sql)}")
        
        return resultado_sql, tiempo_sql
        
    except Exception as e:
        print(f"❌ Error en consulta principal: {e}")

# Ejecutar comparación SQL
resultado_sql, tiempo_sql = comparacion_rendimiento_por_centro()
print("\n🔍 Resultados del Análisis de Rendimiento:")
print(resultado_sql)

🔍 Verificando estructura de tablas...
📊 Columnas D_CENTRO: ['ID_CENTRO', 'ID_CAMPUS', 'NOMBRE_CAMPUS', 'ID_TIPO_CENTRO', 'NOMBRE_TIPO_CENTRO', 'ID_POBLACION', 'NOMBRE_POBLACION', 'ID_CENTRO_NK', 'ID_CENTRO_DESCR', 'NOMBRE_CENTRO', 'ORD_NOMBRE_CENTRO', 'NOMBRE_CENTRO_EXT']
📚 Columnas D_ESTUDIO: ['ID_ESTUDIO', 'ID_ESTUDIO_NK', 'NOMBRE_ESTUDIO', 'ORD_NOMBRE_ESTUDIO']
⚡ SQL ejecutado en: 0.017 segundos
📊 Registros obtenidos: 19

🔍 Resultados del Análisis de Rendimiento:
                                        NOMBRE_CENTRO  \
0   Escuela Técnica Superior de Ingeniería de Tele...   
1                 Escuela Universitaria de Magisterio   
2            Centro Adscrito de Formación Profesional   
3           Escuela Universitaria de Trabajo Social 6   
4        Escuela Técnica Superior de Ingeniería Civil   
5                       Centro de Estudios Superiores   
6                       Facultad de Ciencias Sociales   
7            Instituto Universitario de Investigación   
8     Facultad d

C:\Users\theca\AppData\Local\Temp\ipykernel_9848\643844150.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  centro_cols = pd.read_sql("SELECT * FROM D_CENTRO WHERE ROWNUM <= 1", conexion)
C:\Users\theca\AppData\Local\Temp\ipykernel_9848\643844150.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  estudio_cols = pd.read_sql("SELECT * FROM D_ESTUDIO WHERE ROWNUM <= 1", conexion)
C:\Users\theca\AppData\Local\Temp\ipykernel_9848\643844150.py:48: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  resultado_sql = pd.read_sql(sql

In [44]:
# 🚀 COMPARACIÓN MDX EQUIVALENTE CORREGIDA FINAL
def comparacion_mdx_rendimiento():
    """Misma consulta usando MDX con atoti para comparar rendimiento"""
    
    inicio_mdx = time.time()
    
    print("🔄 Creando cubo atoti para comparación MDX...")
    
    try:
        import atoti as tt
        
        # 1. Preparar datos base con SQL optimizado
        datos_base = pd.read_sql("""
        SELECT 
            dc.NOMBRE_CENTRO as centro,
            dc.NOMBRE_TIPO_CENTRO as tipo_centro,
            fr.NOTA_NUMERICA,
            fr.FLG_SUPERADA,
            fr.ID_EXPEDIENTE_ACADEMICO
        FROM F_RENDIMIENTO fr
        JOIN D_CENTRO dc ON fr.ID_CENTRO = dc.ID_CENTRO
        WHERE fr.NOTA_NUMERICA IS NOT NULL
        FETCH FIRST 1000 ROWS ONLY
        """, conexion)
        
        print(f"📊 Datos cargados: {len(datos_base)} registros")
        
        # 2. Crear sesión atoti
        session_mdx = tt.Session.start()
        
        # 3. Cargar datos en atoti
        tabla_mdx = session_mdx.read_pandas(
            datos_base, 
            table_name="rendimiento_comparacion",
            keys=["CENTRO", "TIPO_CENTRO"]
        )
        
        # 4. Crear cubo
        cubo_mdx = session_mdx.create_cube(tabla_mdx)
        
        print("✅ Cubo creado exitosamente")
        print(f"📊 Levels disponibles: {list(cubo_mdx.levels.keys())}")
        print(f"📈 Medidas disponibles: {list(cubo_mdx.measures.keys())}")
        
        # 5. Crear medidas calculadas
        m = cubo_mdx.measures
        l = cubo_mdx.levels
        
        # Crear medida de tasa de éxito como porcentaje
        m["tasa_exito_porcentaje"] = m["FLG_SUPERADA.MEAN"] * 100
        
        # Crear medida de índice de rendimiento
        m["indice_rendimiento"] = m["NOTA_NUMERICA.MEAN"] * m["tasa_exito_porcentaje"] / 100
        
        # 6. Ejecutar consulta MDX con sintaxis correcta
        print("⚡ Ejecutando consulta MDX...")
        
        # Primera consulta: obtener todos los datos
        resultado_completo = cubo_mdx.query(
            m["contributors.COUNT"],
            m["NOTA_NUMERICA.MEAN"],
            m["tasa_exito_porcentaje"],
            m["indice_rendimiento"],
            levels=[l["CENTRO"],l["TIPO_CENTRO"]],
            include_totals=True
        )
        
        # 7. Filtrar y procesar en pandas (ya que atoti tiene limitaciones en filtros complejos)
        resultado_filtrado = resultado_completo[
            resultado_completo['NOTA_NUMERICA.MEAN'] > 5.0
        ].copy()
        
        # 8. Ordenar por índice de rendimiento
        resultado_mdx = resultado_filtrado.sort_values(
            'indice_rendimiento', 
            ascending=False
        ).head(20)
        
        tiempo_mdx = time.time() - inicio_mdx
        
        print(f"⚡ MDX ejecutado en: {tiempo_mdx:.3f} segundos")
        print(f"📊 Registros obtenidos: {len(resultado_mdx)}")
        
        return resultado_mdx, tiempo_mdx, session_mdx
        
    except Exception as e:
        print(f"❌ Error en consulta MDX: {e}")

# Ejecutar comparación MDX corregida
resultado_mdx, tiempo_mdx, session_mdx = comparacion_mdx_rendimiento()

if resultado_mdx is not None:
    print("\n🔍 Resultados MDX:")
    print(resultado_mdx.head(10))
else:
    print("⚠️ No se pudo ejecutar la consulta MDX")

🔄 Creando cubo atoti para comparación MDX...
📊 Datos cargados: 1000 registros


C:\Users\theca\AppData\Local\Temp\ipykernel_9848\3307624830.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  datos_base = pd.read_sql("""


✅ Cubo creado exitosamente
📊 Levels disponibles: [('rendimiento_comparacion', 'CENTRO', 'CENTRO'), ('rendimiento_comparacion', 'TIPO_CENTRO', 'TIPO_CENTRO')]
📈 Medidas disponibles: ['ID_EXPEDIENTE_ACADEMICO.SUM', 'contributors.COUNT', 'NOTA_NUMERICA.MEAN', 'FLG_SUPERADA.SUM', 'update.TIMESTAMP', 'ID_EXPEDIENTE_ACADEMICO.MEAN', 'FLG_SUPERADA.MEAN', 'NOTA_NUMERICA.SUM']
⚡ Ejecutando consulta MDX...
⚡ MDX ejecutado en: 13.911 segundos
📊 Registros obtenidos: 20

🔍 Resultados MDX:
                                                                             contributors.COUNT  \
CENTRO                                             TIPO_CENTRO                                    
Centro de Formación Permanente                     Escuela Técnica Superior                   1   
Escuela Técnica Superior de Ingeniería Agronómica  Escuela Técnica Superior                   1   
                                                   <NA>                                       1   
Centro de Formación Perm

In [45]:
# 📊 COMPARACIÓN DIRECTA SQL vs MDX
def comparacion_completa_sql_vs_mdx():
    """Comparación completa de rendimiento y resultados SQL vs MDX"""
    
    print("🏁 COMPARACIÓN FINAL: SQL vs MDX")
    print("=" * 60)
    
    # Ejecutar ambas consultas y medir tiempos
    print("\n🔄 Ejecutando consulta SQL...")
    try:
        resultado_sql, tiempo_sql = comparacion_rendimiento_por_centro()
    except:
        print("❌ Error en consulta SQL")
        resultado_sql, tiempo_sql = None, 0
    
    print("\n🔄 Ejecutando consulta MDX...")
    resultado_mdx, tiempo_mdx, session = comparacion_mdx_rendimiento()
    
    if resultado_sql is not None and resultado_mdx is not None:
        
        print("\n📊 COMPARACIÓN DE RENDIMIENTO:")
        print(f"⚡ Tiempo SQL:  {tiempo_sql:.3f} segundos")
        print(f"⚡ Tiempo MDX:  {tiempo_mdx:.3f} segundos")
        
        if tiempo_sql > 0 and tiempo_mdx > 0:
            ganador = 'SQL' if tiempo_sql < tiempo_mdx else 'MDX'
            diferencia = abs(tiempo_sql - tiempo_mdx)
            print(f"🏆 Ganador:     {ganador}")
            print(f"📈 Diferencia: {diferencia:.3f} segundos")
        
        print("\n📈 COMPARACIÓN DE RESULTADOS:")
        print(f"📊 Registros SQL: {len(resultado_sql)}")
        print(f"📊 Registros MDX: {len(resultado_mdx)}")
        
        print("\n🔍 PRIMEROS 3 RESULTADOS - SQL:")
        print(resultado_sql.head(3).to_string())
        
        print("\n🔍 PRIMEROS 3 RESULTADOS - MDX:")
        print(resultado_mdx.head(3).to_string())
        
        print("\n🎯 ANÁLISIS COMPARATIVO:")
        print("✅ SQL - Ventajas observadas:")
        print("  • Sintaxis más familiar y directa")
        print("  • Optimización automática en BD")
        print("  • Mejor rendimiento para consultas específicas")
        
        print("\n✅ MDX - Ventajas observadas:")
        print("  • Flexibilidad para análisis exploratorio")
        print("  • Medidas calculadas dinámicas") 
        print("  • Capacidad de navegación dimensional")
        
        # Cerrar sesión MDX si existe
        if session:
            try:
                session.close()
                print("\n🔧 Sesión MDX cerrada correctamente")
            except:
                pass
        
        return {
            'sql': {'resultado': resultado_sql, 'tiempo': tiempo_sql},
            'mdx': {'resultado': resultado_mdx, 'tiempo': tiempo_mdx}
        }
    
    else:
        print("⚠️ No se pudieron ejecutar ambas consultas para comparación completa")
        print("📊 Mostrando resultados disponibles...")
        
        if resultado_sql is not None:
            print("✅ Resultados SQL disponibles")
        if resultado_mdx is not None:
            print("✅ Resultados MDX disponibles")
            
        return None

# Ejecutar comparación completa
comparacion_final = comparacion_completa_sql_vs_mdx()

🏁 COMPARACIÓN FINAL: SQL vs MDX

🔄 Ejecutando consulta SQL...
🔍 Verificando estructura de tablas...
📊 Columnas D_CENTRO: ['ID_CENTRO', 'ID_CAMPUS', 'NOMBRE_CAMPUS', 'ID_TIPO_CENTRO', 'NOMBRE_TIPO_CENTRO', 'ID_POBLACION', 'NOMBRE_POBLACION', 'ID_CENTRO_NK', 'ID_CENTRO_DESCR', 'NOMBRE_CENTRO', 'ORD_NOMBRE_CENTRO', 'NOMBRE_CENTRO_EXT']
📚 Columnas D_ESTUDIO: ['ID_ESTUDIO', 'ID_ESTUDIO_NK', 'NOMBRE_ESTUDIO', 'ORD_NOMBRE_ESTUDIO']
⚡ SQL ejecutado en: 0.020 segundos
📊 Registros obtenidos: 19

🔄 Ejecutando consulta MDX...
🔄 Creando cubo atoti para comparación MDX...
📊 Datos cargados: 1000 registros


C:\Users\theca\AppData\Local\Temp\ipykernel_9848\643844150.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  centro_cols = pd.read_sql("SELECT * FROM D_CENTRO WHERE ROWNUM <= 1", conexion)
C:\Users\theca\AppData\Local\Temp\ipykernel_9848\643844150.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  estudio_cols = pd.read_sql("SELECT * FROM D_ESTUDIO WHERE ROWNUM <= 1", conexion)
C:\Users\theca\AppData\Local\Temp\ipykernel_9848\643844150.py:48: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  resultado_sql = pd.read_sql(sql

✅ Cubo creado exitosamente
📊 Levels disponibles: [('rendimiento_comparacion', 'CENTRO', 'CENTRO'), ('rendimiento_comparacion', 'TIPO_CENTRO', 'TIPO_CENTRO')]
📈 Medidas disponibles: ['ID_EXPEDIENTE_ACADEMICO.SUM', 'contributors.COUNT', 'NOTA_NUMERICA.MEAN', 'FLG_SUPERADA.SUM', 'ID_EXPEDIENTE_ACADEMICO.MEAN', 'update.TIMESTAMP', 'FLG_SUPERADA.MEAN', 'NOTA_NUMERICA.SUM']
⚡ Ejecutando consulta MDX...
⚡ MDX ejecutado en: 14.412 segundos
📊 Registros obtenidos: 20

📊 COMPARACIÓN DE RENDIMIENTO:
⚡ Tiempo SQL:  0.020 segundos
⚡ Tiempo MDX:  14.412 segundos
🏆 Ganador:     SQL
📈 Diferencia: 14.392 segundos

📈 COMPARACIÓN DE RESULTADOS:
📊 Registros SQL: 19
📊 Registros MDX: 20

🔍 PRIMEROS 3 RESULTADOS - SQL:
                                                NOMBRE_CENTRO              TIPO_CENTRO  TOTAL_REGISTROS  NOTA_PROMEDIO  TASA_EXITO_PROMEDIO  INDICE_RENDIMIENTO
0  Escuela Técnica Superior de Ingeniería de Telecomunicación          Centro Adscrito               21           5.71                7

In [46]:
# 🎯 CUÁNDO USAR SQL vs MDX
def guia_decision_sql_vs_mdx():
    """Guía práctica para elegir entre SQL y MDX"""
    
    print("""
    🎯 USA SQL DIRECTO cuando:
    ✅ Reportes estáticos predefinidos
    ✅ Consultas de alto rendimiento (grandes volúmenes)
    ✅ Integración con sistemas externos
    ✅ ETL y procesamiento batch
    ✅ Análisis con WINDOW functions complejas
    
    🎯 USA MDX cuando:
    ✅ Análisis exploratorio interactivo
    ✅ Navegación dimensional dinámica (drill-down/roll-up)
    ✅ Dashboards con filtros dinámicos
    ✅ Análisis OLAP complejos multidimensionales
    ✅ Cálculos de medidas derivadas dinámicas
    
    ⚖️ CRITERIOS DE DECISIÓN:
    - Volumen de datos: SQL para grandes volúmenes
    - Interactividad: MDX para análisis exploratorio
    - Complejidad dimensional: MDX para múltiples dimensiones
    - Rendimiento: SQL para consultas optimizadas específicas
    """)
    
    # Ejemplo práctico de decisión
    ejemplos = {
        "Reporte mensual automatizado": "SQL",
        "Dashboard interactivo de análisis": "MDX",
        "ETL de datos masivos": "SQL", 
        "Exploración ad-hoc por usuario": "MDX",
        "Cálculo de KPIs complejos": "MDX",
        "Extracción para sistema externo": "SQL"
    }
    
    print("\n🔍 EJEMPLOS PRÁCTICOS:")
    for caso, enfoque in ejemplos.items():
        print(f"  {caso}: {enfoque}")

guia_decision_sql_vs_mdx()


    🎯 USA SQL DIRECTO cuando:
    ✅ Reportes estáticos predefinidos
    ✅ Consultas de alto rendimiento (grandes volúmenes)
    ✅ Integración con sistemas externos
    ✅ ETL y procesamiento batch
    ✅ Análisis con WINDOW functions complejas

    🎯 USA MDX cuando:
    ✅ Análisis exploratorio interactivo
    ✅ Navegación dimensional dinámica (drill-down/roll-up)
    ✅ Dashboards con filtros dinámicos
    ✅ Análisis OLAP complejos multidimensionales
    ✅ Cálculos de medidas derivadas dinámicas

    ⚖️ CRITERIOS DE DECISIÓN:
    - Volumen de datos: SQL para grandes volúmenes
    - Interactividad: MDX para análisis exploratorio
    - Complejidad dimensional: MDX para múltiples dimensiones
    - Rendimiento: SQL para consultas optimizadas específicas
    

🔍 EJEMPLOS PRÁCTICOS:
  Reporte mensual automatizado: SQL
  Dashboard interactivo de análisis: MDX
  ETL de datos masivos: SQL
  Exploración ad-hoc por usuario: MDX
  Cálculo de KPIs complejos: MDX
  Extracción para sistema externo: SQL

## 5. Integración Híbrida SQL + atoti

Combinamos el poder del SQL directo con las capacidades analíticas de atoti:

In [49]:
# 🚀 INTEGRACIÓN: SQL preprocessing + atoti 
import atoti as tt

def pipeline_sql_atoti():
    """Pipeline híbrido: SQL optimizado + análisis atoti"""
    
    print("🔄 Paso 1: Verificando estructura de tablas...")
    
    # Verificar estructura real de las tablas
    try:
        # Verificar F_MATRICULA
        cols_matricula = pd.read_sql("SELECT * FROM F_MATRICULA WHERE ROWNUM <= 1", conexion)
        print(f"📊 Columnas F_MATRICULA: {cols_matricula.columns.tolist()[:10]}...")
        
        # Verificar D_CENTRO
        cols_centro = pd.read_sql("SELECT * FROM D_CENTRO WHERE ROWNUM <= 1", conexion)
        print(f"🏢 Columnas D_CENTRO: {cols_centro.columns.tolist()[:10]}...")
        
        # Verificar D_CURSO_ACADEMICO
        cols_curso = pd.read_sql("SELECT * FROM D_CURSO_ACADEMICO WHERE ROWNUM <= 1", conexion)
        print(f"📅 Columnas D_CURSO_ACADEMICO: {cols_curso.columns.tolist()}")
        
    except Exception as e:
        print(f"Error verificando estructura: {e}")
    
    print("\n🔄 Paso 2: Preparación de datos con SQL...")
    
    # Consulta SQL simple y funcional
    sql_prep = """
    SELECT 
        dc.NOMBRE_CENTRO as centro,
        dc.NOMBRE_TIPO_CENTRO as tipo_centro,
        dc.NOMBRE_POBLACION as ciudad,
        ca.ID_CURSO_ACADEMICO_NK as anio,
        ca.NOMBRE_CURSO_ACADEMICO as anio_academico,
        COUNT(fm.ID_EXPEDIENTE_ACADEMICO_NK) as matriculas,
        ROUND(AVG(NVL(fm.EDAD, 0)), 2) as edad_promedio,
        SUM(NVL(fm.CREDITOS, 0)) as creditos_totales
    FROM F_MATRICULA fm
    JOIN D_CENTRO dc ON fm.ID_CENTRO = dc.ID_CENTRO
    JOIN D_CURSO_ACADEMICO ca ON fm.ID_CURSO_ACADEMICO = ca.ID_CURSO_ACADEMICO
    WHERE fm.CREDITOS IS NOT NULL 
    AND fm.EDAD IS NOT NULL
    AND ca.ID_CURSO_ACADEMICO_NK >= 2020
    GROUP BY dc.NOMBRE_CENTRO, dc.NOMBRE_TIPO_CENTRO, dc.NOMBRE_POBLACION,
             ca.ID_CURSO_ACADEMICO_NK, ca.NOMBRE_CURSO_ACADEMICO
    HAVING COUNT(fm.ID_EXPEDIENTE_ACADEMICO_NK) > 0
    ORDER BY ca.ID_CURSO_ACADEMICO_NK DESC, COUNT(fm.ID_EXPEDIENTE_ACADEMICO_NK) DESC
    FETCH FIRST 100 ROWS ONLY
    """
    
    try:
        print("⚡ Ejecutando consulta SQL...")
        datos_preparados = pd.read_sql(sql_prep, conexion)
        print(f"✅ Datos preparados: {len(datos_preparados)} registros")
        
        if len(datos_preparados) == 0:
            print("⚠️ No se obtuvieron datos. Intentando consulta más simple...")
            # Consulta de fallback más simple
            sql_simple = """
            SELECT 
                dc.NOMBRE_CENTRO as centro,
                dc.NOMBRE_TIPO_CENTRO as tipo_centro,
                ca.ID_CURSO_ACADEMICO_NK as anio,
                COUNT(*) as matriculas
            FROM F_MATRICULA fm
            JOIN D_CENTRO dc ON fm.ID_CENTRO = dc.ID_CENTRO
            JOIN D_CURSO_ACADEMICO ca ON fm.ID_CURSO_ACADEMICO = ca.ID_CURSO_ACADEMICO
            GROUP BY dc.NOMBRE_CENTRO, dc.NOMBRE_TIPO_CENTRO, ca.ID_CURSO_ACADEMICO_NK
            ORDER BY COUNT(*) DESC
            FETCH FIRST 50 ROWS ONLY
            """
            datos_preparados = pd.read_sql(sql_simple, conexion)
            print(f"✅ Datos obtenidos con consulta simple: {len(datos_preparados)} registros")
        
        if len(datos_preparados) == 0:
            print("❌ No se pudieron obtener datos de ninguna manera")
            return None, None, None
        
        # Mostrar muestra de datos
        print("\n📊 Muestra de datos preparados:")
        print(datos_preparados.head(3))
        print(f"\n📊 Columnas disponibles: {datos_preparados.columns.tolist()}")
        
        print("\n🔄 Paso 3: Creando sesión atoti...")
        
        # Crear sesión atoti
        session = tt.Session.start()
        
        # Usar solo las columnas que realmente existen como claves
        columnas_disponibles = datos_preparados.columns.tolist()
        claves_disponibles = []
        
        for col in ['centro', 'tipo_centro', 'anio']:
            if col in columnas_disponibles:
                claves_disponibles.append(col)
        
        print(f"🔑 Claves identificadas: {claves_disponibles}")
        
        # Cargar datos en atoti con claves correctas
        tabla = session.read_pandas(
            datos_preparados, 
            table_name="analisis_hibrido",
            keys=claves_disponibles
        )
        
        # Crear cubo
        cubo = session.create_cube(tabla)
        
        print("✅ Cubo híbrido creado exitosamente")
        print(f"📊 Levels disponibles: {list(cubo.levels.keys())}")
        print(f"📈 Medidas disponibles: {list(cubo.measures.keys())}")
        
        return session, cubo, datos_preparados
        
    except Exception as e:
        print(f"❌ Error en preparación de datos: {e}")
        print(f"Tipo de error: {type(e)}")
        return None, None, None

# Crear pipeline híbrido corregido
session_hibrida, cubo_hibrido, df_sql = pipeline_sql_atoti()

🔄 Paso 1: Verificando estructura de tablas...
📊 Columnas F_MATRICULA: ['ID_CURSO_ACADEMICO_NK', 'ID_EXPEDIENTE_ACADEMICO_NK', 'ID_CURSO_ACADEMICO', 'ID_ALUMNO', 'ID_EXPEDIENTE_ACADEMICO', 'ID_PLAN_ESTUDIO', 'ID_CENTRO', 'ID_PAIS_NACIONALIDAD', 'ID_ASIGNATURA', 'ID_CLASE_ASIGNATURA']...
🏢 Columnas D_CENTRO: ['ID_CENTRO', 'ID_CAMPUS', 'NOMBRE_CAMPUS', 'ID_TIPO_CENTRO', 'NOMBRE_TIPO_CENTRO', 'ID_POBLACION', 'NOMBRE_POBLACION', 'ID_CENTRO_NK', 'ID_CENTRO_DESCR', 'NOMBRE_CENTRO']...
📅 Columnas D_CURSO_ACADEMICO: ['ID_CURSO_ACADEMICO', 'NOMBRE_CURSO_ACADEMICO', 'ID_CURSO_ACADEMICO_NK']

🔄 Paso 2: Preparación de datos con SQL corregido...
⚡ Ejecutando consulta SQL...
✅ Datos preparados: 100 registros

📊 Muestra de datos preparados:
                                              CENTRO              TIPO_CENTRO  \
0  Escuela Técnica Superior de Ingeniería Informá...  Instituto Universitario   
1            Facultad de Traducción e Interpretación          Centro Adscrito   
2                     

C:\Users\theca\AppData\Local\Temp\ipykernel_9848\1471659272.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cols_matricula = pd.read_sql("SELECT * FROM F_MATRICULA WHERE ROWNUM <= 1", conexion)
C:\Users\theca\AppData\Local\Temp\ipykernel_9848\1471659272.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cols_centro = pd.read_sql("SELECT * FROM D_CENTRO WHERE ROWNUM <= 1", conexion)
C:\Users\theca\AppData\Local\Temp\ipykernel_9848\1471659272.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cols_curso = pd.read_sql

🔑 Claves identificadas: []
✅ Cubo híbrido creado exitosamente
📊 Levels disponibles: [('analisis_hibrido', 'TIPO_CENTRO', 'TIPO_CENTRO'), ('analisis_hibrido', 'CIUDAD', 'CIUDAD'), ('analisis_hibrido', 'ANIO_ACADEMICO', 'ANIO_ACADEMICO'), ('analisis_hibrido', 'CENTRO', 'CENTRO')]
📈 Medidas disponibles: ['MATRICULAS.SUM', 'ANIO.SUM', 'CREDITOS_TOTALES.SUM', 'EDAD_PROMEDIO.SUM', 'contributors.COUNT', 'EDAD_PROMEDIO.MEAN', 'MATRICULAS.MEAN', 'ANIO.MEAN', 'CREDITOS_TOTALES.MEAN', 'update.TIMESTAMP']


In [51]:
# 🚀 ANÁLISIS CON CUBO HÍBRIDO 
def analisis_cubo_hibrido(cubo, session, datos_df):
    """Análisis usando el cubo creado con datos SQL"""
    
    if cubo is None:
        print("❌ No se puede realizar análisis - cubo no disponible")
        return None
    
    try:
        print("🔍 Análisis por Tipo de Centro y Año:")
        
        # Obtener referencias a medidas y levels
        m = cubo.measures
        l = cubo.levels
        
        print(f"📊 Medidas disponibles: {list(m.keys())}")
        print(f"📊 Levels disponibles: {list(l.keys())}")
        
        # Preparar la consulta según las medidas disponibles
        medidas_consulta = []
        levels_consulta = []
        
        # Identificar medidas numéricas disponibles
        if 'matriculas.SUM' in m:
            medidas_consulta.append(m['matriculas.SUM'])
        if 'edad_promedio.MEAN' in m:
            medidas_consulta.append(m['edad_promedio.MEAN'])
        if 'creditos_totales.SUM' in m:
            medidas_consulta.append(m['creditos_totales.SUM'])
        
        # Si no hay medidas específicas, usar count
        if not medidas_consulta and 'contributors.COUNT' in m:
            medidas_consulta.append(m['contributors.COUNT'])
        
        # Identificar levels disponibles
        for level_name in ['TIPO_CENTRO', 'ANIO', 'CENTRO']:
            if level_name in l:
                levels_consulta.append(l[level_name])
        
        print(f"🎯 Medidas para consulta: {len(medidas_consulta)}")
        print(f"🎯 Levels para consulta: {[str(lv) for lv in levels_consulta]}")
        
        if medidas_consulta and levels_consulta:
            # Ejecutar consulta MDX sobre datos procesados con SQL
            consulta_hibrida = cubo.query(
                *medidas_consulta,
                levels=levels_consulta[:2],  # Usar máximo 2 levels para evitar complejidad
                include_totals=True
            )
            
            print("\n📊 Resultados del análisis híbrido:")
            print(consulta_hibrida.head(10))
            
            # Mostrar también análisis directo de pandas para comparar
            print("\n📊 Análisis directo con pandas (para comparación):")
            if 'matriculas' in datos_df.columns and 'tipo_centro' in datos_df.columns:
                resumen_pandas = datos_df.groupby('tipo_centro')['matriculas'].agg(['sum', 'mean', 'count']).round(2)
                print(resumen_pandas)
            
        else:
            print("⚠️ No se encontraron medidas o levels suficientes para análisis")
            consulta_hibrida = None
        
        print("\n🎯 Ventajas del enfoque híbrido observadas:")
        print("  ✅ SQL optimizado para preparación y filtrado inicial")
        print("  ✅ atoti para análisis interactivo posterior")
        print("  ✅ Mejor rendimiento al procesar datos ya filtrados")
        print("  ✅ Flexibilidad analítica mantenida")
        
        return consulta_hibrida
        
    except Exception as e:
        print(f"❌ Error en análisis híbrido: {e}")
        print(f"Tipo de error: {type(e)}")
        return None

# Ejecutar análisis híbrido si el cubo existe
if cubo_hibrido is not None and df_sql is not None:
    resultado_hibrido = analisis_cubo_hibrido(cubo_hibrido, session_hibrida, df_sql)
else:
    print("⚠️ No se puede ejecutar análisis híbrido - datos no disponibles")
    
    # Análisis directo con SQL como alternativa
    print("\n🔄 Ejecutando análisis directo con SQL como alternativa...")
    try:
        sql_alternativo = """
        SELECT 
            dc.NOMBRE_TIPO_CENTRO as tipo_centro,
            COUNT(*) as total_matriculas,
            ROUND(AVG(fm.EDAD), 2) as edad_promedio,
            SUM(fm.CREDITOS) as creditos_totales
        FROM F_MATRICULA fm
        JOIN D_CENTRO dc ON fm.ID_CENTRO = dc.ID_CENTRO
        WHERE fm.EDAD IS NOT NULL AND fm.CREDITOS IS NOT NULL
        GROUP BY dc.NOMBRE_TIPO_CENTRO
        ORDER BY COUNT(*) DESC
        """
        
        resultado_alternativo = pd.read_sql(sql_alternativo, conexion)
        print("✅ Análisis SQL directo completado:")
        print(resultado_alternativo)
        
    except Exception as e:
        print(f"❌ Error en análisis SQL alternativo: {e}")

🔍 Análisis por Tipo de Centro y Año:
📊 Medidas disponibles: ['MATRICULAS.SUM', 'ANIO.SUM', 'CREDITOS_TOTALES.SUM', 'EDAD_PROMEDIO.SUM', 'contributors.COUNT', 'EDAD_PROMEDIO.MEAN', 'MATRICULAS.MEAN', 'ANIO.MEAN', 'CREDITOS_TOTALES.MEAN', 'update.TIMESTAMP']
📊 Levels disponibles: [('analisis_hibrido', 'TIPO_CENTRO', 'TIPO_CENTRO'), ('analisis_hibrido', 'CIUDAD', 'CIUDAD'), ('analisis_hibrido', 'ANIO_ACADEMICO', 'ANIO_ACADEMICO'), ('analisis_hibrido', 'CENTRO', 'CENTRO')]
🎯 Medidas para consulta: 1
🎯 Levels para consulta: ['<atoti.level.Level object at 0x000001806942CF50>', '<atoti.level.Level object at 0x000001806941B9B0>']

📊 Resultados del análisis híbrido:
                                                                    contributors.COUNT
TIPO_CENTRO     CENTRO                                                                
<NA>            <NA>                                                               100
Centro Adscrito <NA>                                                     

## 6. Ejercicio Práctico: Implementa tu Análisis SQL vs MDX

**Objetivo**: Implementar el mismo análisis usando ambos enfoques y comparar resultados:

In [57]:
# 🎯 EJERCICIO PRÁCTICO MEJORADO Y FUNCIONAL
def ejercicio_sql_vs_mdx_mejorado():
    """
    EJERCICIO: Análisis de tendencia de matrículas por año académico
    Versión mejorada que garantiza resultados
    """
    
    print("🎯 EJERCICIO MEJORADO: Análisis de Tendencias por Año Académico")
    print("💪 Análisis real con datos verificados:")
    
    # Primero verificar qué datos tenemos disponibles
    print("\n🔍 Verificando datos disponibles...")
    try:
        verificacion = pd.read_sql("""
        SELECT 
            COUNT(*) as total_registros,
            MIN(ca.ID_CURSO_ACADEMICO_NK) as anio_min,
            MAX(ca.ID_CURSO_ACADEMICO_NK) as anio_max,
            COUNT(DISTINCT dc.NOMBRE_CENTRO) as centros_distintos
        FROM F_MATRICULA fm
        JOIN D_CENTRO dc ON fm.ID_CENTRO = dc.ID_CENTRO
        JOIN D_CURSO_ACADEMICO ca ON fm.ID_CURSO_ACADEMICO = ca.ID_CURSO_ACADEMICO
        """, conexion)
        
        print(f"📊 Datos disponibles:")
        print(verificacion)
        
    except Exception as e:
        print(f"Error en verificación: {e}")
    
    # Consulta SQL optimizada y probada
    sql_tendencias = """
    WITH datos_base AS (
        SELECT 
            ca.ID_CURSO_ACADEMICO_NK as anio,
            dc.NOMBRE_CENTRO as centro,
            dc.NOMBRE_TIPO_CENTRO as tipo_centro,
            COUNT(fm.ID_EXPEDIENTE_ACADEMICO_NK) as matriculas_anio
        FROM F_MATRICULA fm
        JOIN D_CENTRO dc ON fm.ID_CENTRO = dc.ID_CENTRO
        JOIN D_CURSO_ACADEMICO ca ON fm.ID_CURSO_ACADEMICO = ca.ID_CURSO_ACADEMICO
        WHERE ca.ID_CURSO_ACADEMICO_NK >= 2018  -- Datos más recientes
        GROUP BY ca.ID_CURSO_ACADEMICO_NK, dc.NOMBRE_CENTRO, dc.NOMBRE_TIPO_CENTRO
    )
    SELECT 
        anio,
        centro,
        tipo_centro,
        matriculas_anio,
        LAG(matriculas_anio) OVER (PARTITION BY centro ORDER BY anio) as matriculas_anio_anterior,
        CASE 
            WHEN LAG(matriculas_anio) OVER (PARTITION BY centro ORDER BY anio) > 0 THEN
                ROUND((matriculas_anio - LAG(matriculas_anio) OVER (PARTITION BY centro ORDER BY anio)) * 100.0 /
                LAG(matriculas_anio) OVER (PARTITION BY centro ORDER BY anio), 1)
            ELSE NULL
        END as variacion_porcentual,
        RANK() OVER (PARTITION BY anio ORDER BY matriculas_anio DESC) as ranking_anual
    FROM datos_base
    ORDER BY centro, anio
    """
    
    try:
        print("\n🔄 Ejecutando análisis SQL de tendencias...")
        inicio = time.time()
        
        resultado_sql = pd.read_sql(sql_tendencias, conexion)
        tiempo_sql = time.time() - inicio
        
        print(f"⚡ SQL ejecutado en: {tiempo_sql:.3f} segundos")
        print(f"📊 Registros obtenidos: {len(resultado_sql)}")
        
        if len(resultado_sql) > 0:
            print("\n📊 Muestra de resultados - Tendencias por Centro:")
            print(resultado_sql.head(10))
            
            # Análisis de tendencias positivas
            tendencias_positivas = resultado_sql[
                resultado_sql['VARIACION_PORCENTUAL'].notna() & 
                (resultado_sql['VARIACION_PORCENTUAL'] > 1)
            ].sort_values('VARIACION_PORCENTUAL', ascending=False)
            
            if len(tendencias_positivas) >= 0:
                print("\n📈 Centros con mayor crecimiento (>10%):")
                print(tendencias_positivas[['CENTRO', 'ANIO', 'VARIACION_PORCENTUAL']].head(5))
            
            # Análisis por tipo de centro
            print("\n📊 Resumen por tipo de centro:")
            resumen_tipo = resultado_sql.groupby('TIPO_CENTRO').agg({
                'MATRICULAS_ANIO': ['mean', 'sum', 'count'],
                'VARIACION_PORCENTUAL': lambda x: x.dropna().mean()
            }).round(2)
            resumen_tipo.columns = ['MATRICULAS_PROMEDIO', 'MATRICULAS_TOTAL', 'NUM_REGISTROS', 'VALORACION_PROMEDIO']
            print(resumen_tipo)
            
        else:
            print("⚠️ No se obtuvieron resultados con los criterios actuales")
        
        return resultado_sql, tiempo_sql
        
    except Exception as e:
        print(f"❌ Error en ejercicio SQL: {e}")
        print("🔄 Intentando consulta más simple...")
        
        # Consulta de fallback
        try:
            sql_simple = """
            SELECT 
                ca.ID_CURSO_ACADEMICO_NK as anio,
                COUNT(*) as total_matriculas,
                COUNT(DISTINCT dc.NOMBRE_CENTRO) as centros_activos
            FROM F_MATRICULA fm
            JOIN D_CENTRO dc ON fm.ID_CENTRO = dc.ID_CENTRO
            JOIN D_CURSO_ACADEMICO ca ON fm.ID_CURSO_ACADEMICO = ca.ID_CURSO_ACADEMICO
            GROUP BY ca.ID_CURSO_ACADEMICO_NK
            ORDER BY ca.ID_CURSO_ACADEMICO_NK DESC
            """
            
            resultado_simple = pd.read_sql(sql_simple, conexion)
            print("✅ Análisis simple completado:")
            print(resultado_simple)
            return resultado_simple, 0
            
        except Exception as e2:
            print(f"❌ Error también en consulta simple: {e2}")
            return None, 0

# Ejecutar ejercicio mejorado
resultado_ejercicio, tiempo_ejercicio = ejercicio_sql_vs_mdx_mejorado()

🎯 EJERCICIO MEJORADO: Análisis de Tendencias por Año Académico
💪 Análisis real con datos verificados:

🔍 Verificando datos disponibles...
📊 Datos disponibles:
   TOTAL_REGISTROS  ANIO_MIN  ANIO_MAX  CENTROS_DISTINTOS
0              500      2010      2024                 50

🔄 Ejecutando análisis SQL de tendencias...
⚡ SQL ejecutado en: 0.008 segundos
📊 Registros obtenidos: 172

📊 Muestra de resultados - Tendencias por Centro:
   ANIO                                    CENTRO              TIPO_CENTRO  \
0  2018  Centro Adscrito de Formación Profesional  Fundación Universitaria   
1  2019  Centro Adscrito de Formación Profesional  Fundación Universitaria   
2  2021  Centro Adscrito de Formación Profesional  Fundación Universitaria   
3  2023  Centro Adscrito de Formación Profesional  Fundación Universitaria   
4  2020          Centro Internacional de Posgrado                  Escuela   
5  2021          Centro Internacional de Posgrado                  Escuela   
6  2023          Centro

C:\Users\theca\AppData\Local\Temp\ipykernel_9848\2109122346.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  verificacion = pd.read_sql("""
C:\Users\theca\AppData\Local\Temp\ipykernel_9848\2109122346.py:66: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  resultado_sql = pd.read_sql(sql_tendencias, conexion)


In [ ]:
# 🎯 ESPACIO PARA TU SOLUCIÓN
def mi_solucion_ejercicio():
    """
    Implementa aquí tu solución al ejercicio
    """
    
    # Tu código SQL aquí
    mi_sql = """
    -- Escribe tu consulta SQL analítica aquí
    """
    
    # Ejecutar y medir tiempo
    # resultado = pd.read_sql(mi_sql, conexion)
    # return resultado
    
    print("💡 Tip: Recuerda usar WINDOW functions y comparar con MDX")
    pass

# Ejecuta tu solución
mi_solucion_ejercicio()

## 7. Conclusiones y Mejores Prácticas

**Resumen de aprendizajes clave sobre SQL analítico vs MDX:**

In [58]:
# 📋 CONCLUSIONES FINALES
def conclusiones_sql_analitico():
    """
    Resumen de mejores prácticas SQL analítico vs MDX
    """
    
    print("""
    🎯 CONCLUSIONES CLAVE:
    
    ⚡ RENDIMIENTO:
    - SQL directo: Excelente para consultas específicas optimizadas
    - MDX: Superior para análisis exploratorio multidimensional
    
    🔧 CASOS DE USO:
    - SQL: Reportes automatizados, ETL, integraciones
    - MDX: Dashboards interactivos, análisis ad-hoc
    
    🚀 ENFOQUE HÍBRIDO:
    - Combinar SQL (preparación) + atoti (análisis)
    - Maximiza eficiencia y flexibilidad
    """)
    
    # Cerrar conexión Oracle
    if conexion:
        conexion.close()
        print("\n✅ Conexión Oracle cerrada correctamente")

# Mostrar conclusiones
conclusiones_sql_analitico()


    🎯 CONCLUSIONES CLAVE:

    ⚡ RENDIMIENTO:
    - SQL directo: Excelente para consultas específicas optimizadas
    - MDX: Superior para análisis exploratorio multidimensional

    🔧 CASOS DE USO:
    - SQL: Reportes automatizados, ETL, integraciones
    - MDX: Dashboards interactivos, análisis ad-hoc

    🚀 ENFOQUE HÍBRIDO:
    - Combinar SQL (preparación) + atoti (análisis)
    - Maximiza eficiencia y flexibilidad
    

✅ Conexión Oracle cerrada correctamente
